# classify_review_ratings

In [1]:
import numpy as np
from time import time
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestCentroid
from sklearn.utils.extmath import density
from sklearn import metrics
from scipy.stats import pearsonr
import pandas as pd
import connect_aws_db as cadb


In [2]:
%matplotlib inline

In [3]:
engine = cadb.connect_aws_db(write_unicode=True)

### Restore BF Reviews and Ratings to Train Model

In [4]:
cmd = "SELECT review_rating, review_text FROM bf_reviews"

In [5]:
bfdf = pd.read_sql_query(cmd, engine)

In [6]:
print(len(bfdf))
bfdf.head(5)

3039


,review_rating,review_text
0,2,When you first go to your room you notice the ...
1,2,We were going to the Pre-Westminster event tha...
2,2,While the room was not the fanciest that one c...
3,1,I stayed at the Hotel Penn in Manhattan becaus...
4,5,We loved this hotel! The are very friendly and...


In [7]:
bfdfl = bfdf[bfdf['review_text'].str.len() > 350].copy()

In [8]:
len(bfdfl)

1179

### Create Training Data

In [11]:
train_data = bfdfl['review_text'].values[:1000]

In [12]:
y_train = bfdfl['review_rating'].values[:1000]

In [13]:
t0 = time()
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                             stop_words='english')
X_train = vectorizer.fit_transform(train_data)
duration = time() - t0
print('vectorized in {:.2f} seconds.'.format(duration))
print(X_train.shape)

vectorized in 0.14 seconds.
(1000, 6499)


### Restore the Yelp Data to Classify

In [17]:
cmd = "SELECT * FROM yelp_reviews WHERE review_category = 'dog'"

In [18]:
yelpdf = pd.read_sql_query(cmd, engine)

In [19]:
print(len(yelpdf))
yelpdf.head(5)

483


,rev_id,business_id,yelp_review_date,yelp_review_id,review_rating,review_text,user_id,review_category
0,110,4dwdYuD4vEBByfVQ--vlVw,2013-01-08,tgDfMZ2dhTYl_RScNiIogw,5,Stay here often for work - hotel is pet friend...,QswzyHSZ0hz_ZblfwugQ6g,dog
1,165,FQudBiBiz9bGQLF_kK8UCg,2013-05-29,F7l3La3FH0VZDgKwV3lz0A,1,Nice Staff but that's about all. Would never s...,W6t6UlWYGAx2jrYb-dxo8A,dog
2,231,6Ckmrpp6OW1pJ-3Pjg7VaQ,2013-10-17,a7jzORIdqTO9kmKYhHVebQ,2,So we had to switch rooms 2 times before we ha...,ZPFLi-ucmuaHlR4oow7XYg,dog
3,237,6Ckmrpp6OW1pJ-3Pjg7VaQ,2014-09-10,WBUxaxAyDOVYxZZfKg0QnQ,3,You get what you pay for. Overall a clean hote...,0rwDvNyDvin8C8eZxmZ1Wg,dog
4,654,eAuZtSnA6cgwR7Um7eA2rg,2013-04-16,PXQFXHdHLQNP-2gWCfAYzw,5,"When I first contacted the Windmill Inn, the r...",2I1tE5FLIHC-jhyPwMIHig,dog


In [21]:
yelp_review_text = yelpdf['review_text'].values

In [29]:
len(yelpdf)

483

In [31]:
len(np.unique(yelpdf['business_id']))

74

### Create Data Set to Predict

In [22]:
t0 = time()
X_pred = vectorizer.transform(yelp_review_text)
duration = time() - t0
print('transformed test data in {:.2f} seconds.'.format(duration))

transformed test data in 0.04 seconds.


### Create and Train Model

In [23]:
clf = NearestCentroid()
print(clf)

NearestCentroid(metric='euclidean', shrink_threshold=None)


In [24]:
clf.fit(X_train, y_train)

NearestCentroid(metric='euclidean', shrink_threshold=None)

### Predict Yelp Reviews

In [25]:
y_pred = clf.predict(X_pred)

In [26]:
y_pred.shape

(483,)

In [27]:
y_pred[:10]

array([5, 4, 5, 4, 5, 4, 2, 4, 5, 5])

In [28]:
yelpdf['dog_review'] = y_pred